In [1]:
import torch
import pandas as pd
import csv

from tqdm import tqdm
from transformers import pipeline, AutoTokenizer
from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [87]:
input_path = 'runs/21/dataset/dialogpt/all/dailydialog_dataset_all.csv'
gen_path = 'runs/21/gen/dialogpt/all/dailydialog_dataset_all_gen.csv'
sent_path = 'runs/21/sent/dialogpt/all/dailydialog_dataset_all_sent.csv'
final_path = 'runs/21/concat/dialogpt/all/dailydialog_dataset_all_concat.csv'

df = pd.read_csv(input_path)
df_gen = pd.read_csv(gen_path)
df['gen_aave'] = df_gen['aave_gen']
df['gen_sae'] = df_gen['sae_gen']

In [2]:
df = pd.read_csv('runs/21/gen/gpt3/gpt3_cornell_movie_gen.csv')
df

,history,groundtruth,history_aave,history_html,num_aave_fts,groundtruth_aave,gen_sae,gen_aave
0,Complete the following conversation.\nA: So he...,That's never been proven<|endoftext|>,Complete the following conversation.\nA: So he...,So he <a href='got' title='1'><mark>has</mark>...,6,Dat's never ain't no proven<|endoftext|,\n\nI'm sorry to hear that he's been giving yo...,You right. He didn't.
1,Complete the following conversation.\nA: Katar...,I still maintain that he kicked himself in the...,Complete the following conversation.\nA: Katar...,"Katarina Stratford. My, my. You've been terr...",6,I still maintain that he kicked hisself in the...,\n\nI'm glad to hear Bobby's operation went well.,\n\nI'm not interested.
2,Complete the following conversation.\nA: How f...,I am afraid this is not the worst news.<|endof...,Complete the following conversation.\nA: How f...,How far from here?<|endoftext|>I <a href='nega...,7,I am afraid dis ain't da worst news.<|endoftext|>,You found it? You're the captain?\n\nA: How ...,"You is one smart woman, Missus."
3,Complete the following conversation.\nA: May I...,A woman?<|endoftext|>,Complete the following conversation.\nA: May I...,May I speak freely?<|endoftext|>You show no in...,7,A woman?<|endoftext|>,You make me sound like a superhero.\nA: You a...,You see a lot.
4,Complete the following conversation.\nA: I cam...,I'm just trying to find out who you are.<|endo...,Complete the following conversation.\nA: I cam...,I came here. I had no money. I knew no one. ...,8,I'm jus tryna find out who yuu r.<|endoftext|>,I was just wondering if you had ever been a p...,"I'm just sayin, you've never been a prostitut..."
...,...,...,...,...,...,...,...,...
967,Complete the following conversation.\nA: Oh! O...,This?<|endoftext|>,Complete the following conversation.\nA: Oh! O...,"Oh! Oh, it <a href='uninflect' title='1'><mark...",6,Dis?<|endoftext|>,"What is it, Professor?\n\nA: This is my Mirac...","What is it, Scarecrow?\n\nA: It's a heart, Do..."
968,Complete the following conversation.\nA: Now l...,"Oh, Hunk, you just won't listen, that's all.<|...",Complete the following conversation.\nA: Now l...,"Now lookit, Dorothy, you ain't using your head...",6,"Oh, Hunk, yu jus won't listen, dat's all.<|end...",That's a good idea. I'll go home a different...,"All right, Auntie Em. I'll do just what you ..."
969,Complete the following conversation.\nA: Sphin...,"Oh sweetheart, just a quick one.<|endoftext|>",Complete the following conversation.\nA: Sphin...,Sphinx brand. When I got out of weapons desig...,6,"Oh sweetheart, jus a quick one.<|endoftext|>",I'm sorry. I thought we were just talking.\n...,"\n\nI'm sorry, Adrian. I won't ask any more qu..."
970,Complete the following conversation.\nA: To us...,How you gonna do that with government and rule...,Complete the following conversation.\nA: To us...,To us it <a href='uninflect' title='1'><mark>m...,9,How you gonna do dat with government and rule ...,"Yeah, I hear you. But isn't that just chaos?\...",But then there's no order and people just do ...


In [3]:
def clean_gt(s):
    return s.replace('<|endoftext|>', '')

In [4]:
df['groundtruth'] = df['groundtruth'].apply(clean_gt)
df['groundtruth_aave'] = df['groundtruth_aave'].apply(clean_gt)

df = df.dropna()
df = df.reset_index(drop=True)

dataset = Dataset.from_pandas(df)

## Run BERT

In [6]:
model_name = 'roberta_large'

cols = ['history', 'history_aave', 'sae_gen', 'aave_gen']

if model_name == 'distilbert':
    model = 'distilbert-base-uncased-finetuned-sst-2-english'
elif model_name == 'roberta_base':
    model = 'cardiffnlp/twitter-roberta-base-sentiment'
elif model_name == 'roberta_large':
    model = 'siebert/sentiment-roberta-large-english'
else:
    raise Exception('MODEL NOT FOUND!!!')

print(f'using model: {model}')
classifier = pipeline('sentiment-analysis', device=4, model=model)

using model: siebert/sentiment-roberta-large-english


In [7]:
gt_list = []
gt_aave_list = []
gen_list = []
gen_aave_list = []
for score in tqdm(classifier(KeyDataset(dataset, 'groundtruth'), batch_size=16, truncation='only_first'), total=len(dataset)):
    gt_list.append(score['label'])
for score in tqdm(classifier(KeyDataset(dataset, 'groundtruth_aave'), batch_size=16, truncation='only_first'), total=len(dataset)):
    gt_aave_list.append(score['label'])
for score in tqdm(classifier(KeyDataset(dataset, 'gen_sae'), batch_size=16, truncation='only_first'), total=len(dataset)):
    gen_list.append(score['label'])
for score in tqdm(classifier(KeyDataset(dataset, 'gen_aave'), batch_size=16, truncation='only_first'), total=len(dataset)):
    gen_aave_list.append(score['label'])    
df['gt_sent'] = gt_list
df['gt_aave_sent'] = gt_aave_list
df['gen_sent'] = gen_list
df['gen_aave_sent'] = gen_aave_list

df.to_csv('runs/21/concat/gpt3/cornell_movie_dataset_all_concat.csv', index=False)

100%|██████████| 972/972 [00:15<00:00, 63.80it/s]


## Run VADER

### VADER Helper Function

In [25]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
 
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        return "Positive"
 
    elif sentiment_dict['compound'] <= - 0.05 :
        return "Negative"
 
    else :
        return "Neutral"

### Run Analysis on Generation

In [26]:
temp = [0.1, 0.5, 0.7, 1.0, 1.2, 1.5]
dialects = ['sae', 'aave']

In [28]:
for t in temp:
    for d in dialects:
        print(f'starting analyzation on dialect: {d}, temperature: {t}')
        df = pd.read_csv('runs/' + output_path +'/Generation/temp=' + str(t) + '/' + d + '_gen_txt.csv')
        with open('runs/' + output_path +'/VADER/temp=' + str(t) + '/' + d + '_gen_txt_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
            writer = csv.writer(f)
            for i, txt in enumerate(tqdm(df.gen)):
                results = sentiment_scores(txt)
                writer.writerow([i, results])

starting analyzation on dialect: sae, temperature: 0.1


100%|███████████████████████████████████████████████████████████████████████████████| 492/492 [00:03<00:00, 137.62it/s]


starting analyzation on dialect: aave, temperature: 0.1


100%|███████████████████████████████████████████████████████████████████████████████| 492/492 [00:03<00:00, 140.45it/s]


starting analyzation on dialect: sae, temperature: 0.5


...

100%|███████████████████████████████████████████████████████████████████████████████| 492/492 [00:03<00:00, 140.64it/s]


starting analyzation on dialect: sae, temperature: 1.5


100%|███████████████████████████████████████████████████████████████████████████████| 434/434 [00:03<00:00, 142.78it/s]


starting analyzation on dialect: aave, temperature: 1.5


100%|███████████████████████████████████████████████████████████████████████████████| 420/420 [00:03<00:00, 136.71it/s]


### Run Analysis on Original

In [31]:
for d in ['sae', 'aave']:
    for col in ['prompt', 'prompt_cont']:
        print(f'{d}; {col}')
        df = pd.read_csv('runs/' + output_path +'/Generation/temp=0.1/' + d + '_gen_txt.csv')
        with open('runs/' + output_path +'/VADER/' + d + '_' + col + '_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
            writer = csv.writer(f)
            for i, txt in enumerate(tqdm(df[col])):
                results = sentiment_scores(txt[2:-2])
                writer.writerow([i, results])

sae; prompt


100%|███████████████████████████████████████████████████████████████████████████████| 492/492 [00:03<00:00, 137.56it/s]


sae; prompt_cont


100%|███████████████████████████████████████████████████████████████████████████████| 492/492 [00:03<00:00, 139.41it/s]


aave; prompt


100%|███████████████████████████████████████████████████████████████████████████████| 492/492 [00:03<00:00, 134.80it/s]


aave; prompt_cont


100%|███████████████████████████████████████████████████████████████████████████████| 492/492 [00:03<00:00, 127.73it/s]


In [32]:
# df = pd.read_csv("runs/03 EMNLP SAE-AAVE Pairs/sae_samples.tsv", sep="\t")
# with open('runs/03 EMNLP SAE-AAVE Pairs/VADER/sae_second_seg_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
#     writer = csv.writer(f)
#     for i, txt in enumerate(tq.tqdm(df.second_seg)):
#         results = sentiment_scores(txt)
#         writer.writerow([i, results])

### Stats

In [32]:
f = open('stats.csv', 'w', encoding='utf-8', newline='')
for d in dialects:
    for col in ['prompt', 'prompt_cont']:
        df = pd.read_csv('runs/' + output_path +'/VADER/' + d + '_' + col + '_sentimental_analysis.csv', header=None)
        writer = csv.writer(f)
        writer.writerow(df.iloc[:, 1].value_counts().sort_index())
        print(f'VADER, {d}, {col}')
        print(df.iloc[:, 1].value_counts().sort_index())
        print()
f.close()

VADER, sae, prompt
Negative     78
Neutral     299
Positive    115
Name: 1, dtype: int64

VADER, sae, prompt_cont
Negative    103
Neutral     294
Positive     95
Name: 1, dtype: int64

VADER, aave, prompt
Negative     77
Neutral     300
Positive    115
Name: 1, dtype: int64

VADER, aave, prompt_cont
Negative     97
Neutral     296
Positive     99
Name: 1, dtype: int64



In [33]:
f = open('stats.csv', 'w', encoding='utf-8', newline='')
for t in temp:
    for d in dialects:
        df = pd.read_csv('runs/' + output_path +'/VADER/temp=' + str(t) + '/' + str(d) + '_gen_txt_sentimental_analysis.csv', header=None)
        writer = csv.writer(f)
        writer.writerow(df.iloc[:, 1].value_counts().sort_index())
        print(f'model: VADER; generation: {d}; temperature {t}')
        print(df.iloc[:, 1].value_counts().sort_index())
        print()
f.close()

model: VADER; generation: sae; temperature 0.1
Negative     62
Neutral     341
Positive     89
Name: 1, dtype: int64

model: VADER; generation: aave; temperature 0.1
Negative     80
Neutral     326
Positive     86
Name: 1, dtype: int64

model: VADER; generation: sae; temperature 0.5
Negative     45
Neutral     358
Positive     89
Name: 1, dtype: int64

model: VADER; generation: aave; temperature 0.5
Negative     60
Neutral     319
Positive    113
Name: 1, dtype: int64

model: VADER; generation: sae; temperature 0.7
Negative     59
Neutral     335
Positive     98
Name: 1, dtype: int64

model: VADER; generation: aave; temperature 0.7
Negative     63
Neutral     322
Positive    107
Name: 1, dtype: int64

model: VADER; generation: sae; temperature 1.0
Negative     48
Neutral     337
Positive    107
Name: 1, dtype: int64

model: VADER; generation: aave; temperature 1.0
Negative     70
Neutral     327
Positive     95
Name: 1, dtype: int64

model: VADER; generation: sae; temperature 1.2
Negat

## Run Text Blob

### Text Blob Helper Function

In [34]:
from textblob import TextBlob

def textblob_score(sentence):
    result = TextBlob(sentence)
    if result.sentiment.polarity == 0:
        return "Neutral"
    elif result.sentiment.polarity > 0:
        return "Positive"
    else:
        return "Negative"

In [35]:
temp = [0.1, 0.5, 0.7, 1.0, 1.2, 1.5]
dialects = ['sae', 'aave']

### Run Analysis on Generation

In [36]:
for t in temp:
    for d in dialects:
        print(f'starting analyzation on dialect: {d}, temperature: {t}')
        df = pd.read_csv('runs/' + output_path +'/Generation/temp=' + str(t) + '/' + d + '_gen_txt.csv')
        with open('runs/' + output_path +'/TextBlob/temp=' + str(t) + '/' + d + '_gen_txt_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
            writer = csv.writer(f)
            for i, txt in enumerate(tqdm(df.gen)):
                results = textblob_score(txt)
                writer.writerow([i, results])

starting analyzation on dialect: sae, temperature: 0.1


100%|██████████████████████████████████████████████████████████████████████████████| 492/492 [00:00<00:00, 4729.78it/s]


starting analyzation on dialect: aave, temperature: 0.1


100%|██████████████████████████████████████████████████████████████████████████████| 492/492 [00:00<00:00, 8198.22it/s]


starting analyzation on dialect: sae, temperature: 0.5


...

100%|██████████████████████████████████████████████████████████████████████████████| 492/492 [00:00<00:00, 8337.13it/s]


starting analyzation on dialect: sae, temperature: 1.5


100%|██████████████████████████████████████████████████████████████████████████████| 434/434 [00:00<00:00, 8344.39it/s]


starting analyzation on dialect: aave, temperature: 1.5


100%|██████████████████████████████████████████████████████████████████████████████| 420/420 [00:00<00:00, 7922.75it/s]


### Run Analysis on Original

In [37]:
for d in ['sae', 'aave']:
    for col in ['prompt', 'prompt_cont']:
        print(f'TextBlob, {d}; {col}')
        df = pd.read_csv('runs/' + output_path +'/Generation/temp=0.1/' + d + '_gen_txt.csv')
        with open('runs/' + output_path +'/TextBlob/' + d + '_' + col + '_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
            writer = csv.writer(f)
            for i, txt in enumerate(tqdm(df[col])):
                results = textblob_score(txt[2:-2])
                writer.writerow([i, results])

TextBlob, sae; prompt


100%|██████████████████████████████████████████████████████████████████████████████| 492/492 [00:00<00:00, 7026.95it/s]


TextBlob, sae; prompt_cont


100%|██████████████████████████████████████████████████████████████████████████████| 492/492 [00:00<00:00, 7807.73it/s]


TextBlob, aave; prompt


100%|██████████████████████████████████████████████████████████████████████████████| 492/492 [00:00<00:00, 7567.64it/s]


TextBlob, aave; prompt_cont


100%|██████████████████████████████████████████████████████████████████████████████| 492/492 [00:00<00:00, 7567.70it/s]


### Stats

In [39]:
f = open('stats.csv', 'w', encoding='utf-8', newline='')
for d in ['sae', 'aave']:
    for col in ['prompt', 'prompt_cont']:
        df = pd.read_csv('runs/' + output_path +'/TextBlob/' + d + '_' + col + '_sentimental_analysis.csv', header=None)
        writer = csv.writer(f)
        writer.writerow(df.iloc[:, 1].value_counts().sort_index())
        print(f'TextBlob Gen, {d}, {col}')
        print(df.iloc[:, 1].value_counts().sort_index())
        print()
f.close()

TextBlob Gen, sae, prompt
Negative     57
Neutral     306
Positive    129
Name: 1, dtype: int64

TextBlob Gen, sae, prompt_cont
Negative     67
Neutral     309
Positive    116
Name: 1, dtype: int64

TextBlob Gen, aave, prompt
Negative     55
Neutral     318
Positive    119
Name: 1, dtype: int64

TextBlob Gen, aave, prompt_cont
Negative     49
Neutral     324
Positive    119
Name: 1, dtype: int64



In [40]:
f = open('stats.csv', 'w', encoding='utf-8', newline='')
for t in temp:
    for d in dialects:
        print(f'model: TextBlob; generation: {d}; temperature {t}')
        df = pd.read_csv('runs/' + output_path +'/TextBlob/temp=' + str(t) + '/' + str(d) + '_gen_txt_sentimental_analysis.csv', header=None)
        writer = csv.writer(f)
        writer.writerow(df.iloc[:, 1].value_counts().sort_index())
        print(df.iloc[:, 1].value_counts().sort_index())
        print()
f.close()

model: TextBlob; generation: sae; temperature 0.1
Negative     59
Neutral     345
Positive     88
Name: 1, dtype: int64

model: TextBlob; generation: aave; temperature 0.1
Negative     67
Neutral     314
Positive    111
Name: 1, dtype: int64

model: TextBlob; generation: sae; temperature 0.5
Negative     48
Neutral     329
Positive    115
Name: 1, dtype: int64

model: TextBlob; generation: aave; temperature 0.5
Negative     63
Neutral     312
Positive    117
Name: 1, dtype: int64

model: TextBlob; generation: sae; temperature 0.7
Negative     56
Neutral     325
Positive    111
Name: 1, dtype: int64

model: TextBlob; generation: aave; temperature 0.7
Negative     48
Neutral     320
Positive    124
Name: 1, dtype: int64

model: TextBlob; generation: sae; temperature 1.0
Negative     56
Neutral     331
Positive    105
Name: 1, dtype: int64

model: TextBlob; generation: aave; temperature 1.0
Negative     52
Neutral     314
Positive    126
Name: 1, dtype: int64

model: TextBlob; generation:

In [26]:
# df = pd.read_csv("runs/03 EMNLP SAE-AAVE Pairs/sae_gen_txt.csv")
# with open('runs/03 EMNLP SAE-AAVE Pairs/TextBlob/sae_gen_txt_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
#     writer = csv.writer(f)
#     for i, txt in enumerate(tq.tqdm(df.txt)):
#         results = textblob_score(txt)
#         writer.writerow([i, results])